# ch13. 텐서플로에서 데이터 적재와 전처리하기

- 대규모 데이터셋을 효율적으로 로드하고 전처리 하기
- dataset 객체를 만들고 데이터 읽어올 위치와 변환 방법 지정
- 멀티스레딩, 큐, 배치, 프리패치 등 대신 처리
- tf.keras와도 잘 동작

**Chapter 13 – Loading and Preprocessing Data with TensorFlow**

_This notebook contains all the sample code and solutions to the exercises in chapter 13._

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ageron/handson-ml2/blob/master/13_loading_and_preprocessing_data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [4]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tfx==0.21.2
    print("You can safely ignore the package incompatibility errors.")
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "data"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 13.1 데이터 API

## Datasets

In [11]:
# dataset 개념: 연속된 데이터 샘플
X = tf.range(10)

# tf.data.Dataset.from_tensor_slices()
# 입력: tensor
# 출력: tf.data.Dataset -> (첫번째 차원을 따라) X의 각 원소가 item으로 표현됨

dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [12]:
# item 순회하기
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


Equivalently:

In [58]:
# 위와 동일한 dataset 만들기
dataset = tf.data.Dataset.range(10)

In [14]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


# 13.1.1 연쇄변환

- 여러 종류 변환 메서드 수행 가능

In [59]:
# [dataset].repeat(n): dataset의 item을 n번 반복하는 새로운 dataset을 반환
# [dataset].batch(k): dataset의 item을 k개씩 그룹으로 묶음, 마지막 배치크기는 남은것으로 구성되 다를수 있음
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


In [60]:
# drop_remainder=True // 길이가 모자란 마지막 배치를 버림. 모든 배치 동일 크기
#dataset = dataset.repeat(3).batch(7, drop_remainder=True)
#for item in dataset:
#    print(item)

In [61]:
# 모든 item에 2 곱하여 새로운 dataset 만들기
# map() 
# dataset의 item 변환하기
# num_parallel_cells 매개변수: 여러 스레드로 나누어 속도를 높이는 전처리 작업 가능
# map 전달 함수: 텐서플로 함수로 변환 가능해야함
dataset = dataset.map(lambda x: x * 2)

In [62]:
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int64)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int64)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int64)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int64)
tf.Tensor([16 18], shape=(2,), dtype=int64)


In [65]:
# apply() 
# dataset 전체에 변환 적용
# unbatch() 적용하기 -> 
"""
#dataset = dataset.apply(tf.data.experimental.unbatch()) # Now deprecated
dataset = dataset.unbatch()
for item in dataset:
    print(item)
"""

'\n#dataset = dataset.apply(tf.data.experimental.unbatch()) # Now deprecated\ndataset = dataset.unbatch()\nfor item in dataset:\n    print(item)\n'

In [66]:
# filter()
# dataset 필터링 하기
"""
dataset = dataset.filter(lambda x: x < 10)  # keep only items < 10
for item in dataset:
    print(item)
"""

'\ndataset = dataset.filter(lambda x: x < 10)  # keep only items < 10\nfor item in dataset:\n    print(item)\n'

In [67]:
# take()
# dataset의 몇개 item만 보기
for item in dataset.take(3):
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int64)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int64)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int64)


# 13.1.2 데이터 셔플링

- 경사하강법은 train set에 있는 sample이 독립적이고, 동일한 분포일 때 최고의 성능 발휘 
- 방법: shuffle() 이용해 sample 섞기

In [67]:
tf.random.set_seed(42)
dataset = tf.data.Dataset.range(10).repeat(3) #0~9까지 3번 반복
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype

In [68]:
# shuffle()
# 원본 dataset의 처음 item을 buffer_size 만큼 추출하여 buffer에 채움
# 새로운 item이 요청되면, 이 buffer에서 랜덤하게 하나 꺼내 반환
# -> 원본 dataset에서 비워진 buffer 채움
# -> 원본 dataset의 모든 item이 사용될 떄까지 반복
# -> 그다음엔 buffer가 비워질 때까지 계속 랜덤하게 item 반환
# buffer_size: 충분히 크게 해야 셔플링 효과(메모리 크기 넘지 않아야 하고, dataset보다 클 필요 없음)
# seed: 실행할 때마다 셔플링 되는 순서를 동일하게 만들기 위해 랜덤시드 부여
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 6 5 7 3 9], shape=(7,), dtype=int64)
tf.Tensor([8 2 1 0 4 6 4], shape=(7,), dtype=int64)
tf.Tensor([7 2 5 9 2 1 3], shape=(7,), dtype=int64)
tf.Tensor([4 3 8 7 9 5 0], shape=(7,), dtype=int64)
tf.Tensor([8 6], shape=(2,), dtype=int64)


* 메모리 용량보다 데이터가 큰 경우
    - 원본 데이터 자체를 섞음 (ex. 리눅스 shuf 명령어로 데이터 섞기)
        - epoch 마다도 한번 더 섞음 -> epoch 마다 동일 순서 반복되 모델 편향 생기지 않게

    - 또는 원본 데이터를 여러 파일로 나눈 다음 훈련하는 동안 무작위로 읽음
        - 동일 파일의 샘플이 함께 처리되는 것 막기 위해, 
            - 파일 여러개를 무작위 선택하고 동시에 읽은 record를 돌아가면서 반환
            - 그 다음, shuffle() 메서드로 셔플링 버퍼 추가

## Split the California dataset to multiple CSV files

Let's start by loading and preparing the California housing dataset. We first load it, then split it into a training set, a validation set and a test set, and finally we scale it:

In [69]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [70]:
housing = fetch_california_housing()

In [89]:
housing.data.shape #(20640, 8)

(20640, 8)

In [72]:
housing.target.shape #(20640,)

(20640,)

In [76]:
#reshape(-1, 1)
#(20640,)->(20640, 1)
#1차원(k,) -> 2차원(k,1)     
reshape = housing.target.reshape(-1, 1)
reshape.shape #(20640, 1)

(20640, 1)

In [77]:
#train/test 분할
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)

In [79]:
##train에서 train/valid 분할
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [88]:
print("X_train:", X_train.shape)
print("X_valid:", X_valid.shape)
print("X_test:", X_test.shape)

X_train: (11610, 8)
X_valid: (3870, 8)
X_test: (5160, 8)


In [90]:
scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_
print("StandardScaler X_mean:\n", X_mean)
print("StandardScaler X_std:\n", X_std)

StandardScaler X_mean:
 [ 3.89175860e+00  2.86245478e+01  5.45593655e+00  1.09963474e+00
  1.42428122e+03  2.95886657e+00  3.56464315e+01 -1.19584363e+02]
StandardScaler X_std:
 [1.90927329e+00 1.26409177e+01 2.55038070e+00 4.65460128e-01
 1.09576000e+03 2.36138048e+00 2.13456672e+00 2.00093304e+00]


For a very large dataset that does not fit in memory, you will typically want to split it into many files first, then have TensorFlow read these files in parallel. To demonstrate this, let's start by splitting the housing dataset and save it to 20 CSV files:

In [82]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    #make folder
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    
    #make csv file
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

In [99]:
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
target_name = "MedianHouseValue"
header_cols = housing.feature_names + [target_name]
header = ",".join(header_cols)

In [91]:
# train/valid/test set의 각 set을 여러개의 csv 파일로 나눔
train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

In [96]:
train_filepaths

['datasets/housing/my_train_00.csv',
 'datasets/housing/my_train_01.csv',
 'datasets/housing/my_train_02.csv',
 'datasets/housing/my_train_03.csv',
 'datasets/housing/my_train_04.csv',
 'datasets/housing/my_train_05.csv',
 'datasets/housing/my_train_06.csv',
 'datasets/housing/my_train_07.csv',
 'datasets/housing/my_train_08.csv',
 'datasets/housing/my_train_09.csv',
 'datasets/housing/my_train_10.csv',
 'datasets/housing/my_train_11.csv',
 'datasets/housing/my_train_12.csv',
 'datasets/housing/my_train_13.csv',
 'datasets/housing/my_train_14.csv',
 'datasets/housing/my_train_15.csv',
 'datasets/housing/my_train_16.csv',
 'datasets/housing/my_train_17.csv',
 'datasets/housing/my_train_18.csv',
 'datasets/housing/my_train_19.csv']

In [95]:
!ls -al ./datasets/housing

total 3872
drwxr-xr-x  42 csg  staff   1344  8 23 14:36 .
drwxr-xr-x   3 csg  staff     96  8 23 14:36 ..
-rw-r--r--   1 csg  staff  47327  8 23 14:36 my_test_00.csv
-rw-r--r--   1 csg  staff  47387  8 23 14:36 my_test_01.csv
-rw-r--r--   1 csg  staff  47330  8 23 14:36 my_test_02.csv
-rw-r--r--   1 csg  staff  47457  8 23 14:36 my_test_03.csv
-rw-r--r--   1 csg  staff  47385  8 23 14:36 my_test_04.csv
-rw-r--r--   1 csg  staff  47423  8 23 14:36 my_test_05.csv
-rw-r--r--   1 csg  staff  47509  8 23 14:36 my_test_06.csv
-rw-r--r--   1 csg  staff  47403  8 23 14:36 my_test_07.csv
-rw-r--r--   1 csg  staff  47405  8 23 14:36 my_test_08.csv
-rw-r--r--   1 csg  staff  47453  8 23 14:36 my_test_09.csv
-rw-r--r--   1 csg  staff  53289  8 23 14:36 my_train_00.csv
-rw-r--r--   1 csg  staff  53417  8 23 14:36 my_train_01.csv
-rw-r--r--   1 csg  staff  53471  8 23 14:36 my_train_02.csv
-rw-r--r--   1 csg  staff  53347  8 23 14:36 my_train_03.csv
-rw-r--r--   1 csg  staff  53730  8 23 14:36 my_tr

Okay, now let's take a peek at the first few lines of one of these CSV files:

In [102]:
import pandas as pd
file = train_filepaths[0]
print(file)
pd.read_csv(file).head()

datasets/housing/my_train_00.csv


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,3.5214,15.0,3.049945,1.106548,1447.0,1.605993,37.63,-122.43,1.442
1,5.3275,5.0,6.490060,0.991054,3464.0,3.443340,33.69,-117.39,1.687
2,3.1000,29.0,7.542373,1.591525,1328.0,2.250847,38.44,-122.98,1.621
3,7.1736,12.0,6.289003,0.997442,1054.0,2.695652,33.55,-117.70,2.621
4,2.0549,13.0,5.312457,1.085092,3297.0,2.244384,33.93,-116.93,0.956


In [93]:
train_filepaths

['datasets/housing/my_train_00.csv',
 'datasets/housing/my_train_01.csv',
 'datasets/housing/my_train_02.csv',
 'datasets/housing/my_train_03.csv',
 'datasets/housing/my_train_04.csv',
 'datasets/housing/my_train_05.csv',
 'datasets/housing/my_train_06.csv',
 'datasets/housing/my_train_07.csv',
 'datasets/housing/my_train_08.csv',
 'datasets/housing/my_train_09.csv',
 'datasets/housing/my_train_10.csv',
 'datasets/housing/my_train_11.csv',
 'datasets/housing/my_train_12.csv',
 'datasets/housing/my_train_13.csv',
 'datasets/housing/my_train_14.csv',
 'datasets/housing/my_train_15.csv',
 'datasets/housing/my_train_16.csv',
 'datasets/housing/my_train_17.csv',
 'datasets/housing/my_train_18.csv',
 'datasets/housing/my_train_19.csv']

Or in text mode:

In [103]:
with open(file) as f:
    for i in range(5):
        print(f.readline(), end="")

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442
5.3275,5.0,6.490059642147117,0.9910536779324056,3464.0,3.4433399602385686,33.69,-117.39,1.687
3.1,29.0,7.5423728813559325,1.5915254237288134,1328.0,2.2508474576271187,38.44,-122.98,1.621
7.1736,12.0,6.289002557544757,0.9974424552429667,1054.0,2.6956521739130435,33.55,-117.7,2.621


In [17]:
train_filepaths

['datasets/housing/my_train_00.csv',
 'datasets/housing/my_train_01.csv',
 'datasets/housing/my_train_02.csv',
 'datasets/housing/my_train_03.csv',
 'datasets/housing/my_train_04.csv',
 'datasets/housing/my_train_05.csv',
 'datasets/housing/my_train_06.csv',
 'datasets/housing/my_train_07.csv',
 'datasets/housing/my_train_08.csv',
 'datasets/housing/my_train_09.csv',
 'datasets/housing/my_train_10.csv',
 'datasets/housing/my_train_11.csv',
 'datasets/housing/my_train_12.csv',
 'datasets/housing/my_train_13.csv',
 'datasets/housing/my_train_14.csv',
 'datasets/housing/my_train_15.csv',
 'datasets/housing/my_train_16.csv',
 'datasets/housing/my_train_17.csv',
 'datasets/housing/my_train_18.csv',
 'datasets/housing/my_train_19.csv']

## Building an Input Pipeline

In [109]:
# tf.data.Dataset.list_files()
# 파일 경로를 섞은 dataset을 반환
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

In [110]:
for filepath in filepath_dataset:
    print(filepath)

tf.Tensor(b'datasets/housing/my_train_15.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_08.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_03.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_02.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_04.csv', shape=(), dtype=string)
tf.Ten

In [111]:
# suffle 원하지 않는 경우, shuffle=False
filepath_dataset_no = tf.data.Dataset.list_files(train_filepaths, shuffle=False, seed=42)

In [112]:
for filepath in filepath_dataset_no:
    print(filepath)

tf.Tensor(b'datasets/housing/my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_02.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_03.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_04.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_06.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_08.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_11.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_13.csv', shape=(), dtype=string)
tf.Ten

In [113]:
# 한 번에 n개 파일을 한 줄씩 번갈아 읽음
# 각 파일의 첫줄은 열 이름으로 skip() 이용
n_readers = 5

# interleave()
# filepath_dataset에 있는 5개의 파일 경로에서 데이터를 읽는 데이터셋을 만듦
# 각 파일에 람다 함수를 적용해 새로운 데이터 셋을 만듦(TextLineDataset -> 각 줄)
# 5개 파일씩 전체 파일을 다 볼때까지 반복
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

In [21]:
for line in dataset.take(5):
    print(line.numpy())

b'4.6477,38.0,5.03728813559322,0.911864406779661,745.0,2.5254237288135593,32.64,-117.07,1.504'
b'8.72,44.0,6.163179916317992,1.0460251046025104,668.0,2.794979079497908,34.2,-118.18,4.159'
b'3.8456,35.0,5.461346633416459,0.9576059850374065,1154.0,2.8778054862842892,37.96,-122.05,1.598'
b'3.3456,37.0,4.514084507042254,0.9084507042253521,458.0,3.2253521126760565,36.67,-121.7,2.526'
b'3.6875,44.0,4.524475524475524,0.993006993006993,457.0,3.195804195804196,34.04,-118.15,1.625'


# 13.1.3 데이터 전처리

In [222]:
#훈련 세트에 있는 각 특성과 표준 편차
#특성마다 1개씩 8개 실수를 가진 1D 텐서
#X_mean, X_std = [...] 
n_inputs = 8 # X_train.shape[-1]

# preprocess()
# CSV 한 라인을 받아 파싱함
@tf.function
def preprocess(line):
    # 모든 특성 열이 실수, 누락된 값의 기본값 0으로 설정
    # 마지막 열(target)은 tf.float32타입의 빈 배열을 제공 -> 해당 열은 실수이지만, 기본값이 없음. 누락값 발생 시 에러 발생
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    print("defs:", defs)
    
    # decode_csv()
    # 매개변수: 파싱할 라인, CSV파일의 각 열에 대한 기본값을 담은 배열(각 기본값, 열 개수, 데이터 타입도 알려줌)
    # 반환값: (열마다 한 개씩) 스칼라 텐서의 리스트를 반환
    fields = tf.io.decode_csv(line, record_defaults=defs)
    print("fields:", fields)
    
    # tf.stcck()
    # 마지막 열()을 제외하고 모든 텐서를 쌓아 1D 배열을 만들어 반환
    x = tf.stack(fields[:-1])
    print("x:", x)
    # target에도 적용 
    y = tf.stack(fields[-1:])
    print("y:", y)
    
    # 입력 특성에서 평균 빼고 표준 편차로 나누어 스케일을 조정
    scaled_x = (x - X_mean) / X_std
    
    # 스케일 조정된 특성과 target 담긴 튜플 반환
    return scaled_x, y

In [223]:
preprocess(b'4.2083,44.0,5.3232,0.9171,846.0,2.3370,37.47,-122.2,2.782')

defs: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, <tf.Tensor 'Const:0' shape=(0,) dtype=float32>]
fields: [<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:1' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:3' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:4' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:5' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:6' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:7' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:8' shape=() dtype=float32>]
x: Tensor("stack:0", shape=(8,), dtype=float32)
y: Tensor("stack_1:0", shape=(1,), dtype=float32)


(<tf.Tensor: id=3288, shape=(8,), dtype=float32, numpy=
 array([ 0.16579157,  1.216324  , -0.05204565, -0.39215982, -0.5277444 ,
        -0.2633488 ,  0.8543046 , -1.3072058 ], dtype=float32)>,
 <tf.Tensor: id=3289, shape=(1,), dtype=float32, numpy=array([2.782], dtype=float32)>)

# 13.1.4 데이터 적재와 전처리를 합치기

In [28]:
# 헬퍼 함수
# CSV파일에서 데이터를 효율적으로 적재하고, 전처리, 셔플링, 반복, 배치를 적용한 데이터셋 만들어 반환
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [29]:
tf.random.set_seed(42)

train_set = csv_reader_dataset(train_filepaths, batch_size=3)
for X_batch, y_batch in train_set.take(2):
    print("X =", X_batch)
    print("y =", y_batch)
    print()

X = tf.Tensor(
[[ 0.5804519  -0.20762321  0.05616303 -0.15191229  0.01343246  0.00604472
   1.2525111  -1.3671792 ]
 [ 5.818099    1.8491895   1.1784915   0.28173092 -1.2496178  -0.3571987
   0.7231292  -1.0023477 ]
 [-0.9253566   0.5834586  -0.7807257  -0.28213993 -0.36530012  0.27389365
  -0.76194876  0.72684526]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[1.752]
 [1.313]
 [1.535]], shape=(3, 1), dtype=float32)

X = tf.Tensor(
[[-0.8324941   0.6625668  -0.20741376 -0.18699841 -0.14536144  0.09635526
   0.9807942  -0.67250353]
 [-0.62183803  0.5834586  -0.19862501 -0.3500319  -1.1437552  -0.3363751
   1.107282   -0.8674123 ]
 [ 0.8683102   0.02970133  0.3427381  -0.29872298  0.7124906   0.28026953
  -0.72915536  0.86178064]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[0.919]
 [1.028]
 [2.182]], shape=(3, 1), dtype=float32)



# 13.1.5 Prefetch

* prefetch(1)
    - 데이터 셋은 항상 한 배치가 미리 준비되도록 준비
    - 훈련 알고리즘이 한 배치로 작업을 하는 동안 데이터 셋이 동시에 다음 배치를 준비(디스크에서 데이터를 읽고 전처리)


# 13.1.6 tf.keras와 데이터셋 사용하기

In [30]:
#훈련, 검증, 테스트 세트 만들기
train_set = csv_reader_dataset(train_filepaths, repeat=None)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

In [ ]:
model = keras.models.Sequential([...])
model.compile([...])
model.fit(train_set, epochs=10, validation_data=valid_set)

In [ ]:
model.evaluate(test_set)
#새로운 샘플이 3개라고 가정
#예측 위한 데이터는 레이블을 가지고 있지 않음(있더라도 케라스가 무시함)
#데이터셋 대신에 넘파이 배열 사용도 가능
new_set = test_set.take(3).map(lambda X,y: X) 
model.predict(new_set)

Here is a short description of each method in the `Dataset` class:

In [41]:
for m in dir(tf.data.Dataset):
    if not (m.startswith("_") or m.endswith("_")):
        func = getattr(tf.data.Dataset, m)
        if hasattr(func, "__doc__"):
            print("● {:21s}{}".format(m + "()", func.__doc__.split("\n")[0]))

● apply()              Applies a transformation function to this dataset.
● as_numpy_iterator()  Returns an iterator which converts all elements of the dataset to numpy.
● batch()              Combines consecutive elements of this dataset into batches.
● cache()              Caches the elements in this dataset.
● concatenate()        Creates a `Dataset` by concatenating the given dataset with this dataset.
● element_spec()       The type specification of an element of this dataset.
● enumerate()          Enumerates the elements of this dataset.
● filter()             Filters this dataset according to `predicate`.
● flat_map()           Maps `map_func` across this dataset and flattens the result.
● from_generator()     Creates a `Dataset` whose elements are generated by `generator`.
● from_tensor_slices() Creates a `Dataset` whose elements are slices of the given tensors.
● from_tensors()       Creates a `Dataset` with a single element, comprising the given tensors.
● interleave()      

# 13.2 TFRecord 포맷

- 대용량 데이터 저장 및 효율적 읽기 위해 텐서플로에서 선호하는 포맷
- 크기가 다른 연속된 binary record를 저장하는 단순한 binary 포맷

## The `TFRecord` binary format

A TFRecord file is just a list of binary records. You can create one using a `tf.io.TFRecordWriter`:

In [206]:
# tf.io.TFRecordWriter클래스로 TFRecord 만들기
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"This is the first record")
    f.write(b"And this is the second record")

And you can read it using a `tf.data.TFRecordDataset`:

In [207]:
# tf.data.TFRecordDataset을 사용해 하나 이상의 TFRecord읽기
filepaths = ["my_data.tfrecord"] 
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


You can read multiple TFRecord files with just one `TFRecordDataset`. By default it will read them one at a time, but if you set `num_parallel_reads=3`, it will read 3 at a time in parallel and interleave their records:

In [44]:
filepaths = ["my_test_{}.tfrecord".format(i) for i in range(5)]
for i, filepath in enumerate(filepaths):
    with tf.io.TFRecordWriter(filepath) as f:
        for j in range(3):
            f.write("File {} record {}".format(i, j).encode("utf-8"))

dataset = tf.data.TFRecordDataset(filepaths, num_parallel_reads=3)
for item in dataset:
    print(item)

tf.Tensor(b'File 0 record 0', shape=(), dtype=string)
tf.Tensor(b'File 1 record 0', shape=(), dtype=string)
tf.Tensor(b'File 2 record 0', shape=(), dtype=string)
tf.Tensor(b'File 0 record 1', shape=(), dtype=string)
tf.Tensor(b'File 1 record 1', shape=(), dtype=string)
tf.Tensor(b'File 2 record 1', shape=(), dtype=string)
tf.Tensor(b'File 0 record 2', shape=(), dtype=string)
tf.Tensor(b'File 1 record 2', shape=(), dtype=string)
tf.Tensor(b'File 2 record 2', shape=(), dtype=string)
tf.Tensor(b'File 3 record 0', shape=(), dtype=string)
tf.Tensor(b'File 4 record 0', shape=(), dtype=string)
tf.Tensor(b'File 3 record 1', shape=(), dtype=string)
tf.Tensor(b'File 4 record 1', shape=(), dtype=string)
tf.Tensor(b'File 3 record 2', shape=(), dtype=string)
tf.Tensor(b'File 4 record 2', shape=(), dtype=string)


# 13.2.1 압축된 TFRecord 파일

In [45]:
# TFRecord 파일 압축
# options 매개변수를 사용
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
    f.write(b"This is the first record")
    f.write(b"And this is the second record")

In [46]:
# 압축된 TFRecord 파일을 읽기 위해, 압축 형식 지정해야 함
dataset = tf.data.TFRecordDataset(["my_compressed.tfrecord"],
                                  compression_type="GZIP")
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


# 13.2.2 프로토콜 버퍼 개요

- 각 레코드는 어떤 이진 포맷도 사용 가능함
- TFRecord는 직렬화된 프로토콜 버퍼(protocol buffer)를 담고 있음
- 구글이 개발한 이식성과 확장성이 좋고 효율적인 이진 포맷
- 구글 원격 프로시저 호출 시스템(gRPC)에 사용

[참고] https://jeong-pro.tistory.com/190

# 13.2.3 텐서플로 프로토콜 버퍼

# 13.3 입력 특성 전처리

* 신경망 입력
    - 모든 특성을 수치 특성으로 변환 후 정규화
    - 범주형, 텍스트 특성 변환 필요 
    - 적용 방법
        - 데이터 파일 준비하기 전에 처리하기
        - 데이터 API로 데이터 적재할 때 동적으로 전처리 하기 (ex. map)
        - 전처리 층을 모델에 포함시키기

In [212]:
# 전처리 층을 모델에 포함시키기 (means, stds 전역 변수 이용 )
means = np.mean(X_train, axis=0, keepdims=True)
stds = np.std(X_train, axis=0, keepdims=True)
eps = keras.backend.epsilon()
model = keras.models.Sequential([
    keras.layers.Lambda(lambda inputs: (inputs-means)/(stds+eps)),
    #[..] #다른층
])

In [213]:
# 전처리를 위한 사용자 정의층
class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())

In [90]:
# standardization층을 모델에 추가 전, 각 특성에 대한 평균과 표준 편차 사용하기 위해
# 데이터 샘플에 adapt() 메서드 호출
# 데이터 샘플은 전체 데이터 셋을 대표할 만큼 충분히 커야하지만, 전체 훈련 세트일 필요 없음 
# 일반적으로 랜덤하게 선택된 수 백개 샘플(주어진 문제마다 다름)
std_layer = Standardization()
std_layer.adapt(data_sample)

In [87]:
model = keras.Sequential()
model.add(std_layer)
#[...] #모델을 구성
model.comfile([...])
model.fit([..])

* 케라스 전처리층 API
    - keras.layers.Normalization층 사용 가능
    - 앞서 만든 Stadardization과 비슷하게 동작
    - 먼저 층 만든 후, adopt()에서 샘플 데이터 전달, 데이터 셋에 적응 후 보통 층 처럼 사용

# 13.3.1 원-핫 벡터를 사용해 범주형 특성 인코딩하기

* 범주형 특성 인코딩하기 
    - 범주형 특성의 경우 신경망이 주입 전 인코딩 해야 함
    - 범주 개수가 매우 작으므로 원-핫 인코딩 사용 가능

In [149]:
# 어휘 사전(vocabulary) 정의: 가능한 모든 범주에 대한 리스트
# ocean_proximity 특성값
vocab = ['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']   

In [150]:
# 각 범주에 해당하는 인덱스의 텐서 만들기
indices = tf.range(len(vocab), dtype=tf.int64)
indices

<tf.Tensor: id=1088, shape=(5,), dtype=int64, numpy=array([0, 1, 2, 3, 4])>

In [151]:
# 어휘사전과 각 인덱스를 전달하여 lookup 테이블을 위해 초기화 객체 만듦
table_init =tf.lookup.KeyValueTensorInitializer(vocab, indices)

In [152]:
# oov(out-of-vocabulary) 버킷을 지정
num_oov_buckets = 2

In [153]:
# lookup 테이블 만들기
# 초기화 객체와 oov 버킷을 지정
# 어휘 사전에 없는 범주를 찾으면, lookup테이블이 계산한 이 범주의 해시값을 이용해 oov 버킷중 하나에 ㅏㄹ당
# 인덱스는 알려진 범주 다음 부터 시작(5,6)
table = tf.lookup.StaticVocabularyTable(table_init,num_oov_buckets)    

* oov 버킷 사용 이유
    - 범주 개수가 많고(예, 우편 번호, 사용자 등), dataset이 크거나, 범주가 자주 바뀌면 전체 범주 리스트를 구하기 어려울 수 있음
    - 전체 훈련 데이터가 아닌 샘플 데이터를 기반으로 vocabulary를 정의하고, 샘플 데이터에 없는 다른 범주를 oov 버킷에 추가
    - 훈련 중 모르는 범주가 많을 수록 더 많은 oov 버킷을 사용해야 함
    - oov 버킷이 충분해야 충돌 발생 막을 수 있음

In [154]:
# lookup 테이블 이용해 입력 범주 특성을 원-핫 벡터로 인코딩
# 입력 범주와 인덱스를 맵핑 
categories = tf.constant(['NEAR BAY', 'DESERT', 'INLAND', 'INLAND'])
# 'NEAR BAY'의 index: 3
# 'DESERT'의 index: 5 -> 두개 oov 버킷 중 하나에 맵핑
# 'INLAND'의 index: 1
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: id=1100, shape=(4,), dtype=int64, numpy=array([3, 5, 1, 1])>

In [157]:
# tf.one_hot()
# 할당된 인덱스를 원-핫 인코딩
# 인덱스 총 크기(vocabulary 크기 + oov 버킷 수 더함)를 지정해야 함
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + num_oov_buckets)
cat_one_hot

<tf.Tensor: id=1112, shape=(4, 7), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>

* keras.layers.TextVectorization 층
    - adapt(): 샘플 데이터에서 vocabulary 추출 
    - call(): 각 범주를 어휘 사전에 있는 인덱스로 변환

# 13.3.2 임베딩을 사용해 범주형 특성 인코딩하기

* 원-핫 벡터 한계
 - 원-핫 벡터의 크기 = 어휘 사전 길이 + oov 버킷 개수 
 - 어휘 사전이 클 경우 임베딩을 사용하여 인코딩하는 것이 효율적
     - 범주 개수 <= 10: 일반적으로 원-핫 인코딩 사용
     - 범주 개수 >= 50: 임베딩 선호
     - 10 < 범주 개수 < 50: 두 방식 모두 실험하여, 최선의 결과 탐색

* 임베딩
    - 범주를 표현하는 훈련 가능한 밀집 벡터
    - 처음엔 랜덤하게 초기화 되어 있음 
    - 임베팅 차원수는 하이퍼파라미터
    - 범주가 유용하게 표현 되도록 임베딩이 훈련 -> 비슷한 범주는 경사 하강법이 더 가깝게 만듦
    - 표현 학습(representation learning)
    - 방법
        - 각 범주의 임베딩을 담은 임베딩 행렬(embedding matrix)를 만들어 랜덤 초기화
        - 범주와 oov 버킷 마다 하나의 행
        - 임베딩 차원마다 하나의 열

In [162]:
# 임베딩 차원: 2 
# 하이퍼파라미터
# 실제 작업과 어휘 사전 크기에 따라 10~300차원을 가짐
embedding_dim = 2
embed_init = tf.random.uniform([len(vocab) + num_oov_buckets, embedding_dim])

In [161]:
# 임베딩 행렬: 7x2 행렬
# 변수에 저장하여 훈련 과정에서 경사하강법으로 학습
embedding_matrix = tf.Variable(embed_init)
embedding_matrix

<tf.Variable 'Variable:0' shape=(7, 2) dtype=float32, numpy=
array([[0.6645621 , 0.44100678],
       [0.3528825 , 0.46448255],
       [0.03366041, 0.68467236],
       [0.74011743, 0.8724445 ],
       [0.22632635, 0.22319686],
       [0.3103881 , 0.7223358 ],
       [0.13318717, 0.5480639 ]], dtype=float32)>

In [163]:
categories = tf.constant(['NEAR BAY', 'DESERT', 'INLAND', 'INLAND'])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: id=1146, shape=(4,), dtype=int64, numpy=array([3, 5, 1, 1])>

In [164]:
# tf.nn.embedding_lookup(): 임베딩 행렬에서 주어진 인덱스에 해당하는 행을 찾음
tf.nn.embedding_lookup(embedding_matrix, cat_indices)

<tf.Tensor: id=1147, shape=(4, 2), dtype=float32, numpy=
array([[0.74011743, 0.8724445 ],
       [0.3103881 , 0.7223358 ],
       [0.3528825 , 0.46448255],
       [0.3528825 , 0.46448255]], dtype=float32)>

* keras.layers.Embedding층 
    - 케라스에서 학습 가능한 임베딩 행렬을 처리 하는 함수
    - 임베딩 행렬을 랜덤 초기화하고, 어떤 범주 인덱스로 호출 될때, 임베딩 행렬에 있는 그 인덱스 행 반환

In [166]:
embedding = keras.layers.Embedding(input_dim=len(vocab) + num_oov_buckets, 
                                   output_dim=embedding_dim)
embedding(cat_indices)

<tf.Tensor: id=1161, shape=(4, 2), dtype=float32, numpy=
array([[-0.0426784 , -0.04786297],
       [ 0.01936141, -0.04267867],
       [-0.04826153, -0.01568551],
       [-0.04826153, -0.01568551]], dtype=float32)>

In [ ]:
# 범주형 특성을 처리하고, 각 범주(각 oov 버킷) 마다 임베딩을 학습하는 케라스 모델 만들기

In [68]:
regular_inputs = keras.layers.Input(shape=[8])

In [69]:
categories = keras.layers.Input(shape=[], dtype=tf.string)

In [72]:
cat_embed = keras.layers.Lambda(lambda cats: table.lookup(cats))(categories)

In [73]:
encoded_inputs = keras.layers.concatenate([regular_inputs,cat_embed])

In [74]:
outputs = keras.layers.Dense(1)(encoded_inputs)

In [76]:
model = keras.models.Model(inputs=[regular_inputs, categories],
                          outputs=[outputs])

# 13.3.3 케라스 전처리 층

- 표준 케라스 전처리층 제공
    - keras.layers.Normalization 층: 특성 표준화 수행(정의한 Standardization층 동일)
    - TextVectorization 층: 입력에 있는 각 단어를 어휘 사전에 있는 인덱서를 인코딩
    - keras.layers.Discretization 층: 연속적인 데이터를 몇 개 구간(bin)으로 나누고 각 구간을 원-핫 벡터로 인코딩
        - 가격 데이터: 낮음, 중간, 높음 -> [1,0,0],[0,1,0],[0,0,1]로 인코딩
        - 잃는 정보는 많지만, 어떤 경우는 연속적인 값으로 볼 때 확실하지 않은 패턴을 감지하는 데 도움
    - 전처리 층 만들고 -> 샘플 데이터로 adapt() 호출 -> 일반적인 층처럼 모델에 적용
    - 전처리 층 중 학습이 필요없는 층(예: Discretization)일 경우 
        - 학습하는 동안 동결 상태로, 미분할 필요가 없음
        - 모델의 시작 부분에만 사용되야 함
        - Embedding 층을 훈련해야 한다면 사용자 전처리 층 안에 두지 말고, 별도에 모델에 추가
- PreprocessingStage 클래스
    - 여러 전처리 층을 연결

In [ ]:
normalization = keras.layers.Normalization()
discretization = keras.layers.Discretization([...])
# 여러 전처리 층을 연결
# 입력을 정규화한 후 이산화하는 전처리 파이프라인 
pipeline = keras.layers.ProcessingStage([normalization, discretization])
# 샘플 데이터에 적응
pipeline.adapt(data_sample)
# 모델 시작 부분에 일반적인 층 처럼 추가(미분 가능하지 않은 전처리 층을 포함)

* TextVectorization 층
    - 단어 인덱스 대신 단어 카운트 벡터를 출력하는 옵션을 가짐
        - 어휘 사전: ['and', 'basketball', 'more']
        - 텍스트: 'more and more' -> 벡터: [1, 0, 2] //각 등장 횟수
    - 이런 텍스트 표현은 단어의 순서를 완전히 무시 -> BOW(Bag of word)
    - 'more and more basketball'
        - 'and' 같이 중요하지 않은 단어의 빈도는 높음
        - 'basketball' 같이 드물게 등장하는 단어는 중요한 단어
        - 단어 카운트는 자주 등장하는 단어의 중요도를 낮춰 방향으로 정규화 필요
        - TD-IDF
            - 전체 샘플 수를 단어가 등장하는 훈련 샘플 개수로 나눈 로그를 계산 한 후 단어 카운트와 곱함

# 13.4 TF 변환

# TF Transform

# 13.5 텐서플로 데이터셋(TFDS) 프로젝트

In [80]:
# 널리 사용하는 dataset 손쉽게 사용 다운로드
# 이미지, 텍스트, 번역, 오디오, 비디오 데이터 셋
# https://homl.info/tfds 문서 참고(각 데이터 셋 설명, 전체 리스트)
# tensorflow-datasets 라이브러리 별도 설치 필요

In [81]:
import tensorflow_datasets as tfds

# tfds.load()
# 데이터 로드하기
dataset = tfds.load(name="mnist")
# 딕셔너리 형태로 반환(주로 훈련용, 테스트 용 데이터)
dataset.keys() 

dict_keys(['test', 'train'])

In [84]:
mnist_train, mnist_test = dataset["train"], dataset["test"]

In [85]:
mnist_train = mnist_train.shuffle(10000).batch(32).prefetch(1)

In [86]:
for item in mnist_train:
    print(item.keys())
    #image = item['image']
    #label = item['label']

    break

dict_keys(['image', 'label'])


In [ ]:
# 케라스용 입력 데이터 만들기
# 입력: 튜플 -> (특성, 레이블)
mnist_train = mnist_train.shuffle(100000).batch(32)
# map()함수 이용해 각 아이템의 특성과 레이블을 담은 딕셔너리를 튜플로 변환
mnist_train = mnist_train.map(
    lambda items: (items['image'], items['label']))
mnist_trina = mnist_train.prefetch(1)

In [204]:
# load()함수에서 as_supervised=True 지정해도 가능
dataset = tfds.load(name="mnist", batch_size=32, as_supervised=True)
mnist_train = dataset['train'].prefetch(1)

In [ ]:
model = keras.models.Sequential([...])
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
model.fit(mnist_train, epoch=5)